In [1]:
import os,sys,shutil
import tensorflow as tf
import numpy as np
import argparse
import cv2,math,glob,random,time
import time
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import random

In [0]:
# Model
REGULARIZER_COF = 1e-6

def _fc_variable( weight_shape,name="fc"):
    with tf.variable_scope(name):
        # check weight_shape
        input_channels  = int(weight_shape[0])
        output_channels = int(weight_shape[1])
        weight_shape    = (input_channels, output_channels)

        # define variables
        weight = tf.get_variable("w", weight_shape     , initializer=tf.contrib.layers.xavier_initializer())
        bias   = tf.get_variable("b", [weight_shape[1]], initializer=tf.constant_initializer(0.0))
    return weight, bias

def _conv_variable( weight_shape,name="conv"):
    with tf.variable_scope(name):
        # check weight_shape
        w = int(weight_shape[0])
        h = int(weight_shape[1])
        input_channels  = int(weight_shape[2])
        output_channels = int(weight_shape[3])
        weight_shape = (w,h,input_channels, output_channels)
        # define variables
        weight = tf.get_variable("w", weight_shape     , initializer=tf.contrib.layers.xavier_initializer_conv2d())
        bias   = tf.get_variable("b", [output_channels], initializer=tf.constant_initializer(0.0))
    return weight, bias

def _deconv_variable( weight_shape,name="conv"):
    with tf.variable_scope(name):
        # check weight_shape
        w = int(weight_shape[0])
        h = int(weight_shape[1])
        output_channels = int(weight_shape[2])
        input_channels  = int(weight_shape[3])
        weight_shape = (w,h,input_channels, output_channels)
        regularizer = tf.contrib.layers.l2_regularizer(scale=REGULARIZER_COF)
        # define variables
        weight = tf.get_variable("w", weight_shape    ,
                                initializer=tf.contrib.layers.xavier_initializer_conv2d(),
                                regularizer=regularizer)
        bias   = tf.get_variable("b", [input_channels], initializer=tf.constant_initializer(0.0))
    return weight, bias

def _conv2d( x, W, stride):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

def _deconv2d( x, W, output_shape, stride=1):
    # x           : [nBatch, height, width, in_channels]
    # output_shape: [nBatch, height, width, out_channels]
    return tf.nn.conv2d_transpose(x, W, output_shape=output_shape, strides=[1,stride,stride,1], padding = "SAME",data_format="NHWC")


def _conv_layer(x, input_layer, output_layer, stride, filter_size=3, name="conv", isTraining=True):
    conv_w, conv_b = _conv_variable([filter_size,filter_size,input_layer,output_layer],name="conv"+name)
    h = _conv2d(x,conv_w,stride=stride) + conv_b
    h = tf.contrib.layers.batch_norm(h, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=isTraining, scope="gNormc"+name)
    h = tf.nn.leaky_relu(h)
    return h

def _deconv_layer(x,input_layer, output_layer, stride=2, filter_size=3, name="deconv", isTraining=True):
    bs, h, w, c = x.get_shape().as_list()
    deconv_w, deconv_b = _deconv_variable([filter_size,filter_size,input_layer,output_layer],name="deconv"+name )
    h = _deconv2d(x,deconv_w, output_shape=[bs,h*2,w*2,output_layer], stride=stride) + deconv_b
    h = tf.contrib.layers.batch_norm(h, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=isTraining, scope="gNormd"+name)
    h = tf.nn.leaky_relu(h)
    return h

def buildGenerator(x,reuse=False,isTraining=True,nBatch=64,resBlock=4,name="generator"):

    with tf.variable_scope(name, reuse=reuse) as scope:
        if reuse: scope.reuse_variables()

        h = _conv_layer(x, 3, 64, 1, 7 , "i-1_g")

        tmp = h

        for i in range(resBlock):
            conv_w, conv_b = _conv_variable([3,3,64,64],name="res%s-1" % i)
            nn = _conv2d(h,conv_w,stride=1) + conv_b
            nn = tf.contrib.layers.batch_norm(nn, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=isTraining, scope="Normr%s-1_g" %i)
            nn = tf.nn.leaky_relu(nn)
            conv_w, conv_b = _conv_variable([3,3,64,64],name="res%s-2" % i)
            nn = _conv2d(nn,conv_w,stride=1) + conv_b
            nn = tf.contrib.layers.batch_norm(nn, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=isTraining, scope="Normr%s-2_g" %i)

            nn = tf.math.add(h,nn, name="resadd%s" % i)
            h = nn

        conv_w, conv_b = _conv_variable([3,3,64,64],name="conv3-1_g" )
        h = _conv2d(h,conv_w,stride=1) + conv_b
        h = tf.contrib.layers.batch_norm(h, decay=0.9, updates_collections=None, epsilon=1e-5, scale=True, is_training=isTraining, scope="Norm3-1_g")
        h = tf.math.add(tmp,h, name="add")

        h = _conv_layer(h, 64, 64, 1, 3 , "3-2_g")


        conv_w, conv_b = _conv_variable([7,7,64,3],name="convo_g" )
        h = _conv2d(h,conv_w,stride=1) + conv_b
        y = tf.nn.tanh(h)

    return y


In [0]:
# Data Generator
class BatchGenerator:
    def __init__(self, img_size, datadir):
        self.folderPath = datadir
        #self.orgSize = (218,173)
        self.imgSize = (img_size,img_size)
        assert self.imgSize[0]==self.imgSize[1]


    def add_mosaic(self,img,ocp):
        #flg = True
        occupancy = ocp#np.random.uniform(min_occupancy, max_occupancy)
        mosaic =img
        h, w, _ = img.shape
        img_for_cnt = np.zeros((h, w), np.uint8)

        while True:
            ms_size = np.random.randint(8,16)
            scale = np.random.randint(4,8)
            x = random.randint(0, max(0, w - 1 - ms_size))
            y = random.randint(0, h - 1 - ms_size)
            area = img[y:y+ms_size,x:x+ms_size]
            area = cv2.resize(area,(ms_size//scale,ms_size//scale), interpolation=cv2.INTER_NEAREST)
            area = cv2.resize(area,(ms_size,ms_size), interpolation=cv2.INTER_NEAREST)
            mosaic[y:y+ms_size,x:x+ms_size] = area
            area = np.where(area>0, 255, area)
            img_for_cnt[y:y+ms_size,x:x+ms_size] = cv2.cvtColor(area, cv2.COLOR_RGB2GRAY)
            #print((img_for_cnt > 0).sum())
            if (img_for_cnt > 0).sum() > h * w * occupancy:

                break
        return mosaic

    def add_impulse(self, img, ocp):
        rate = np.random.rand() * ocp/2 + ocp/2
        mask = np.random.binomial(size=img.shape, n=1, p=rate)
        noise = np.random.randint(256, size=img.shape)
        img = img * (1 - mask) + noise * mask
        return img


    def add_noise(self, img, ocp):
        #noise = (np.random.rand(self.imgSize[0],self.imgSize[1],3) -0.5) * 512
        rate = np.random.rand() * ocp/2 + ocp/2
        noise = np.random.normal(0,100,(self.imgSize[0],self.imgSize[1],3))
        occupy = np.random.binomial(n=1,p=rate,size=[self.imgSize[0],self.imgSize[1],3])
        noise = noise * occupy
        img = img+noise
        img = np.clip(img,0,255)
        return img

    def strongNoise(self, img, ntype="gauss"):
        height = img.shape[0]
        width = img.shape[1]
        #print(height,width)
        noise = np.zeros_like(img)

        if ntype == "random":
            noise = (np.random.rand(height,width,3) - 0.5) * 255

        elif ntype == "gauss":
            noise = (np.random.randn(height,width,3) - 0.5) * 255

        elif ntype == "sandp":
            noise = (np.random.rand(height,width) - 0.5) * 255
            noise = np.where( noise > 255/4, 255, 0)
            noise2 =(np.random.rand(height,width) - 0.5) * 255
            noise2 = np.where( noise2 > 255/4, 0, -255)
            noise += noise2
            noise = np.tile(noise,3).reshape(height,width,3)

        new = img + noise
        return new

    def getBatch(self, nBatch, id, subFolder, ocp=0.5):
        x   = np.zeros( (nBatch,self.imgSize[0],self.imgSize[1],3), dtype=np.float32)
        imagePath = glob.glob(self.folderPath + "/" + subFolder + "/*")
        for i,j in enumerate(id):
            if j >= len(imagePath):
                continue
            img = cv2.imread(imagePath[j])
            dmin = min(img.shape[0],img.shape[1])
            img = img[int(0.5*(img.shape[0]-dmin)):int(0.5*(img.shape[0]+dmin)),int(0.5*(img.shape[1]-dmin)):int(0.5*(img.shape[1]+dmin)),:]
            img = cv2.resize(img,self.imgSize)

            #img = self.add_noise(img,ocp)
            #img = self.add_mosaic(img,ocp)
            #img = self.add_impulse(img,ocp)


            x[i,:,:,:] = (img - 127.5) / 127.5 

        return x

In [0]:
def loss_g(y, t):
    #mse = tf.reduce_mean(tf.square(y - t))
    loss = tf.nn.l2_loss(y-t) + tf.reduce_sum(tf.abs(y-t))
    return loss


def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01,
                                       beta1=0.9,
                                       beta2=0.999)
    train_step = optimizer.minimize(loss)
    return train_step

def tileImage(imgs):
    d = int(math.sqrt(imgs.shape[0]-1))+1
    h = imgs[0].shape[0]
    w = imgs[0].shape[1]
    r = np.zeros((h*d,w*d,3),dtype=np.float32)
    for idx,img in enumerate(imgs):
        idx_y = int(idx/d)
        idx_x = idx-idx_y*d
        r[idx_y*h:(idx_y+1)*h,idx_x*w:(idx_x+1)*w,:] = img
    return r

def foloderLength(folder):
    dir = folder
    paths = os.listdir(dir)
    return len(paths)

def printParam(scope):
    total_parameters = 0
    for variable in tf.trainable_variables(scope=scope):
        # shape is an array of tf.Dimension
        shape = variable.get_shape()
        variable_parameters = 1
        for dim in shape:
            variable_parameters *= dim.value
        total_parameters += variable_parameters
    print("{} has {} parameters".format(scope, total_parameters))

In [5]:
from google.colab import drive
drive.mount('/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /drive


In [6]:
!ls "/drive/My Drive/ML_Project"
ROOT_PATH = "/drive/My Drive/ML_Project/"
DATASET_DIR = ROOT_PATH + "data"
VAL_DIR = ROOT_PATH + "val"
SAVE_DIR = ROOT_PATH + "checkpoints"
SVIM_DIR = ROOT_PATH + "samples"

checkpoints  data  Noise2Noise.ipynb  processed  samples  val


In [0]:
def main():
    if not os.path.exists(SAVE_DIR):
        os.makedirs(SAVE_DIR)
    if not os.path.exists(SVIM_DIR):
        os.makedirs(SVIM_DIR)

    img_size = 256
    bs = 16

    dir = DATASET_DIR
    val = VAL_DIR
    datalen = foloderLength(DATASET_DIR)
    vallen = foloderLength(VAL_DIR)

    # loading images on training
    batch = BatchGenerator(img_size=img_size,datadir=dir)
    val = BatchGenerator(img_size=img_size,datadir=val)

    id = np.random.choice(range(datalen),bs)
    IN_ = tileImage(batch.getBatch(bs,id, 'a')[:4])

    IN_ = (IN_ + 1)*127.5
    cv2.imwrite("input.png",IN_)


    start = time.time()

    x = tf.placeholder(tf.float32, [bs, img_size, img_size, 3])
    t = tf.placeholder(tf.float32, [bs, img_size, img_size, 3])

    y =buildGenerator(x,nBatch=bs)

    loss = loss_g(y, t)
    printParam(scope="generator")

    train_step = training(loss)

    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    saver = tf.train.Saver()
    summary = tf.summary.merge_all()

    ckpt = tf.train.get_checkpoint_state(SAVE_DIR)

    if ckpt: # checkpointがある場合
        last_model = ckpt.model_checkpoint_path # 最後に保存したmodelへのパス
        print ("load " + last_model)
        saver.restore(sess, last_model) # 変数データの読み込み
        print("succeed restore model")
    else:
        print("models were not found")
        init = tf.global_variables_initializer()
        sess.run(init)

    print("%.4e sec took initializing"%(time.time()-start))


    hist =[]


    start = time.time()
    for i in range(100000):
        # loading images on training
        id = np.random.choice(range(datalen),bs)
        batch_images_x = batch.getBatch(bs,id,'a', ocp=0.5,)
        batch_images_t = batch.getBatch(bs,id,'b', ocp=0.5)

        tmp, yloss = sess.run([train_step,loss], feed_dict={
            x: batch_images_x,
            t: batch_images_t
        })

        print("in step %s loss = %.4e" %(i,yloss))
        hist.append(yloss)

        if i %40 ==0:
            id = np.random.choice(range(vallen),bs)
            batch_images_x = val.getBatch(bs,id,'a',ocp=0.5)
            out = sess.run(y,feed_dict={
                x:batch_images_x})
            X_ = tileImage(batch_images_x[:4])
            Y_ = tileImage(out[:4])

            X_ = (X_ + 1)*127.5
            Y_ = (Y_ + 1)*127.5
            Z_ = np.concatenate((X_,Y_), axis=1)
            #print(np.max(X_))
            cv2.imwrite("{}/{}.png".format(SVIM_DIR,i),Z_)

            fig = plt.figure()
            ax = fig.add_subplot(111)
            plt.title("Loss")
            plt.grid(which="both")
            plt.yscale("log")
            ax.plot(hist,label="test", linewidth = 0.5)
            plt.savefig("hist.png")
            plt.close()

            print("%.4e sec took per 100steps" %(time.time()-start))
            start = time.time()

        if i%40==0 and i > 10 :
            if i>1900:
                loss_1k_old = np.mean(hist[-2000:-1000])
                loss_1k_new = np.mean(hist[-1000:])
                print("old loss=%.4e , new loss=%.4e"%(loss_1k_old,loss_1k_new))
                if loss_1k_old*2 < loss_1k_new:
                    break

            saver.save(sess,os.path.join(SAVE_DIR,"model.ckpt"),i)

In [0]:
main()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

generator has 389571 parameters
load /drive/My Drive/ML_Project/checkpoints/model.ckpt-25000
INFO:tensorflow:Restoring parameters from /drive/My Drive/ML_Project/checkpoints/model.ckpt-25000
succeed restore model
8.1891e+00 sec took initializing
in step 0 loss = 1.6079e+06
6.7642e+00 sec took per 100steps
in step 1 loss = 1.1036e+06
in step 2 loss = 9.5873e+05
in step 3 loss = 6.8545e+05
in step 4 loss = 1.0840e+06
in step 5 loss = 9.6718e+05
in step 6 loss = 1.1877e+06
in step 7 loss = 6.2766e+05
in step 8 loss = 1.1945e+06
in step 9 loss = 6.7328e+05
in step 10 loss = 1.6479e+06
in step 11 loss = 1.1661e+06
in step 12 loss = 9